In [2]:
from pykrx import stock
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
warnings.filterwarnings("ignore")
import math

In [3]:
#public 5/31-6/21 data 가져오기
# tickers = stock.get_market_ticker_list("20230531", market="ALL")
# daily = pd.DataFrame(tickers, columns=['티커'])

# for s in tqdm(daily['티커']):
#     price = stock.get_market_ohlcv("20230531", "20230621", s, adjusted=False)
#     price = price.reset_index()
    
#     for idx, row in price.iterrows():
#         date_str = row['날짜'].strftime('%Y-%m-%d')
#         daily.loc[daily['티커'] == s, date_str] = row['종가']

# daily = daily.set_index('티커')
# daily.to_csv('daily_adjusted.csv')

In [4]:
#일별 종가 data 저장
daily = pd.read_csv('daily_adjusted.csv')
daily = daily.reset_index()
daily.drop('index',axis=1,inplace=True)
daily['티커'] = 'A' + daily['티커']
daily

,티커,2023-05-31,2023-06-01,2023-06-02,2023-06-05,2023-06-07,2023-06-08,2023-06-09,2023-06-12,2023-06-13,2023-06-14,2023-06-15,2023-06-16,2023-06-19,2023-06-20,2023-06-21
0,A060310,3020.0,2900.0,2875.0,2935.0,2890.0,2960.0,2965.0,3075.0,3040.0,3000.0,3025.0,2960.0,3070.0,3005.0,2985.0
1,A095570,4255.0,4300.0,4425.0,4440.0,4390.0,4380.0,4415.0,4380.0,4390.0,4405.0,4415.0,4680.0,4630.0,4545.0,4550.0
2,A006840,18830.0,18670.0,18920.0,19080.0,19000.0,18810.0,18740.0,18230.0,18290.0,19170.0,18870.0,19590.0,19800.0,19680.0,19110.0
3,A054620,11160.0,11100.0,10980.0,10760.0,11100.0,10740.0,11050.0,10980.0,10870.0,10690.0,10720.0,10520.0,10390.0,10230.0,10040.0
4,A265520,21350.0,21550.0,21600.0,21250.0,21350.0,21400.0,21550.0,21350.0,21750.0,21500.0,21800.0,21750.0,21500.0,21450.0,21400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714,A000547,17240.0,17370.0,16760.0,16090.0,14630.0,15000.0,14370.0,13450.0,12140.0,11010.0,10110.0,10100.0,9280.0,7510.0,9760.0
2715,A000545,5590.0,5560.0,5670.0,5760.0,5690.0,5680.0,5500.0,5450.0,5530.0,5890.0,5790.0,5680.0,5680.0,5670.0,5910.0
2716,A003280,1352.0,1342.0,1362.0,1380.0,1385.0,1391.0,1449.0,1530.0,1496.0,1447.0,1420.0,1411.0,1451.0,1886.0,2115.0
2717,A037440,9190.0,9170.0,9220.0,9230.0,9220.0,9070.0,9100.0,9110.0,9070.0,8900.0,8830.0,8910.0,9160.0,9180.0,9300.0


In [5]:
#연율화된 매매일의 일간수익률 (변동성 계산 위함)
returns = daily.copy()
returns.set_index('티커', inplace=True)
daily_returns = pd.DataFrame(index=returns.index)
daily_returns_short = pd.DataFrame(index=returns.index)
for i in range(1, len(returns.columns)):
    prev_col = returns.columns[i-1]
    curr_col = returns.columns[i]
    perc_col = i+1
    
    daily_returns[perc_col] = (returns[curr_col] - returns[prev_col]) / returns[prev_col] * 250
    daily_returns_short[perc_col] = (returns[curr_col] - returns[prev_col]) / returns[prev_col] *250 *-1
 
daily_returns=daily_returns.reset_index()
daily_returns_short=daily_returns_short.reset_index()
daily_returns #연율화된 일간 수익률

,티커,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,A060310,-9.933775,-2.155172,5.217391,-3.833049,6.055363,0.422297,9.274874,-2.845528,-3.289474,2.083333,-5.371901,9.290541,-5.293160,-1.663894
1,A095570,2.643948,7.267442,0.847458,-2.815315,-0.569476,1.997717,-1.981880,0.570776,0.854214,0.567537,15.005663,-2.670940,-4.589633,0.275028
2,A006840,-2.124270,3.347616,2.114165,-1.048218,-2.500000,-0.930356,-6.803629,0.822820,12.028431,-3.912363,9.538951,2.679939,-1.515152,-7.240854
3,A054620,-1.344086,-2.702703,-5.009107,7.899628,-8.108108,7.216015,-1.583710,-2.504554,-4.139834,0.701590,-4.664179,-3.089354,-3.849856,-4.643206
4,A265520,2.341920,0.580046,-4.050926,1.176471,0.585480,1.752336,-2.320186,4.683841,-2.873563,3.488372,-0.573394,-2.873563,-0.581395,-0.582751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714,A000547,1.885151,-8.779505,-9.994033,-22.684897,6.322625,-10.500000,-16.005567,-24.349442,-23.270181,-20.435967,-0.247280,-20.297030,-47.683190,74.900133
2715,A000545,-1.341682,4.946043,3.968254,-3.038194,-0.439367,-7.922535,-2.272727,3.669725,16.274864,-4.244482,-4.749568,0.000000,-0.440141,10.582011
2716,A003280,-1.849112,3.725782,3.303965,0.905797,1.083032,10.424155,13.975155,-5.555556,-8.188503,-4.664824,-1.584507,7.087172,74.948312,30.355249
2717,A037440,-0.544070,1.363141,0.271150,-0.270856,-4.067245,0.826902,0.274725,-1.097695,-4.685777,-1.966292,2.265006,7.014590,0.545852,3.267974


In [6]:
# ohlcv = stock.get_market_ohlcv("20230531", market="ALL")
# ohlcv2 = stock.get_market_ohlcv("20230621", market="ALL")

# private_data = pd.concat([ohlcv['종가'],ohlcv2['종가']],axis=1)
# private_data = private_data.reset_index()


#종목별 최종 수익률
private_data = daily.iloc[:,[0,1,-1]]
private_data = private_data.dropna()
private_data.columns = ['티커','종가_531', '종가_621']

private_data['매수수익률'] = (private_data['종가_621'] - private_data['종가_531'])/private_data['종가_531']
private_data['매도수익률'] = (private_data['종가_621'] - private_data['종가_531'])/private_data['종가_531'] *-1

private_data

,티커,종가_531,종가_621,매수수익률,매도수익률
0,A060310,3020.0,2985.0,-0.011589,0.011589
1,A095570,4255.0,4550.0,0.069330,-0.069330
2,A006840,18830.0,19110.0,0.014870,-0.014870
3,A054620,11160.0,10040.0,-0.100358,0.100358
4,A265520,21350.0,21400.0,0.002342,-0.002342
...,...,...,...,...,...
2714,A000547,17240.0,9760.0,-0.433875,0.433875
2715,A000545,5590.0,5910.0,0.057245,-0.057245
2716,A003280,1352.0,2115.0,0.564349,-0.564349
2717,A037440,9190.0,9300.0,0.011970,-0.011970


## sharp 지수 계산

##### Sharpe index = (연율화된 총자산 최종 수익률 - 무위험 수익률) / 총자산 일간 수익률 변동성

In [13]:
train = pd.read_csv('../submission_lstm_ver2.csv') #체크하고 싶은 submission 파일 올리기
train = train.drop('Unnamed: 0',axis=1)
train.columns = ['티커','순위']
merged_return = train.merge(private_data, on='티커')
# merged_return.to_csv('valid.csv')

In [14]:
merged_return = merged_return.sort_values(by='순위')
train = train.sort_values(by='순위')

In [15]:
#연율화된총자산최종수익율 
returns = (merged_return['매수수익률'][:200].sum() + merged_return['매도수익률'][1800:].sum())/400
annualized_return = returns * 250 /15  
annualized_return

-0.20482374490090724

In [16]:
#상위 200 / 하위200 종목 

merged_vol_long = train.merge(daily_returns, on='티커')
merged_vol_short = train.merge(daily_returns_short, on='티커')

# merged_vol_long['연율화된 일간수익률 평균'] = merged_vol_long.iloc[:,2:].mean(axis=1)
# merged_vol_short['연율화된 일간수익률 평균'] = merged_vol_long.iloc[:,2:].mean(axis=1)

merged_long_short = pd.concat([merged_vol_long[0:200], merged_vol_short[1800:]], ignore_index=True)


#연율화된 일간 수익률의 평균
annualized_daily = []

for i in range(2,16):
    avg = merged_long_short.iloc[:,i].mean()
    annualized_daily.append(avg)
    
annualized_total = sum(annualized_daily)/14

#총자산 일간 수익률 변동성 계산
nom= 0
for i in range(len(annualized_daily)):
    nom += (annualized_daily[i]-annualized_total)**2

vol = math.sqrt(nom/13)

In [17]:
sharpe_index = (annualized_return - 0.035) / vol
print("sharpe_index: ", sharpe_index)

sharpe_index:  -0.1176117204295844
